In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [3]:
!pip install selenium
!pip install webdriver_manager

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time

# Crawling

## 동적크롤링

reference: https://selenium-python.readthedocs.io/index.html

### 설치 방법

```cmd
pip install selenium
```

Chrome Driver: https://chromedriver.chromium.org/downloads

### 사용법

기본적인 사용 방법은 아래와 같음
```python
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
 
driver = webdriver.Chrome(executable_path='chromedriver') # executable_path: chromedriver path
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))    # 4버전 이상
# driver = webdriver.Firefox()

driver.get(url="http://www.naver.com")

driver.close()
```

In [23]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://finance.daum.net/domestic/market_cap')
bs = BeautifulSoup(driver.page_source, 'lxml')


In [26]:
bs.select('div.box_contents')

[<div class="box_contents"><div style="display: block;"><table>
 <colgroup>
 <col width="44"/>
 <col width="*"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="84"/>
 <col width="94"/>
 <col width="84"/>
 </colgroup>
 <thead>
 <tr>
 <th class="lAlign">순위</th>
 <th class="lAlign">종목명</th>
 <th class="rAlign">현재가</th>
 <th class="rAlign">전일비</th>
 <th class="rAlign">
 <a class="" data-field-name="changeRate" href="javascript:void(0)">등락률 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="marketCap" href="javascript:void(0)">시가총액 <i>-</i></a>
 </th>
 <th class="rAlign">
 <a class="" data-field-name="listedShareCount" href="javascript:void(0)">상장주식수 <i>-</i></a>
 </th>
 <th class="rAlign last">
 <a class="" data-field-name="foreignOwnRate" href="javascript:void(0)">외국인 <i>-</i></a>
 </th>
 </tr>
 </thead>
 <tbody><tr class="first"><td class="first"><span class="time">1</span></td>
 <td><a class="txt" href="/quotes/A005930">삼성전자</a></td>
 <td>
 <sp

#### Locating Elements

find_element: 조건에 해당하는 하나의 값 반환  
find_elements: 조건에 해당하는 다수의 값 반환

element를 찾는 method는 아래와 같음
- by_ID
- by_xpath
- by_link_text
- by_partial_link_text
- by_name
- by_tag_name
- by_class_name
- by_css_selector

```python
element = driver.find_element(By.XPATH, x_path)
element = driver.find_element_by_xpath(x_path)

element = driver.find_element(By.CSS_SELECTOR, css)
element = driver.find_element_by_css_selector(css)
```

해당 값을 못 찾은 경우 NoSuchElementException 발생

In [14]:
import time



# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[7] #71 -> 72
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[8]
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[11] #75 -> 76
# /html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[11]

In [24]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get('https://finance.daum.net/domestic/market_cap')
time.sleep(1)
stock_info = []

columns = [tag.text.strip().replace(' -', '').replace('+', '') for tag in bs.select('div.box_contents')[0].select('tr')[0].select('th')]
driver.find_element(By.XPATH, '/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[11]').click()
time.sleep(1)

driver.find_element(By.XPATH, '/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[7]').click()
time.sleep(1)
bs = BeautifulSoup(driver.page_source, 'lxml')

total_stock_info = []
for row in range(1, 31):
    stock_info.append([tag.text.strip().replace('▼', '').replace('-', '').replace('%', '').replace('▲', '').replace('+', '') for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])
total_stock_info.extend(stock_info)

for page in tqdm(range(7, 12)):
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    time.sleep(2)
    bs = BeautifulSoup(driver.page_source, 'lxml')
    if page < 11:
        for row in range(1, 31):
            stock_info.append([tag.text.strip().replace('▼', '').replace('-', '').replace('%', '').replace('▲', '').replace('+', '') for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])
    else:
         for row in range(1, 26):
            stock_info.append([tag.text.strip().replace('▼', '').replace('-', '').replace('%', '').replace('▲', '').replace('+', '') for tag in bs.select('div.box_contents')[0].select('tr')[row].select('td')])

total_stock_info.extend(stock_info)
total_stock_info

100%|██████████| 5/5 [00:10<00:00,  2.16s/it]


[['2101', 'KOSEF 독일DAX', '12,045', '205', '1.73', '60', '500,000', '0.00'],
 ['2102',
  'KB 인버스 2X S&P 500 선물 ETN',
  '6,020',
  '130',
  '2.11',
  '60',
  '1,000,000',
  '0.00'],
 ['2103', 'RISE 국채선물10년', '54,420', '200', '0.37', '60', '110,000', '0.00'],
 ['2104',
  'TIGER 200커뮤니케이션서비스',
  '16,610',
  '145',
  '0.88',
  '60',
  '360,000',
  '0.00'],
 ['2105',
  '한투 인버스 2X S&P500 선물 ETN(H)',
  '1,194',
  '26',
  '2.13',
  '60',
  '5,000,000',
  '0.00'],
 ['2106',
  'KODEX MSCI EM선물(H)',
  '8,500',
  '60',
  '0.71',
  '60',
  '700,000',
  '0.00'],
 ['2107', 'RISE TDF2030액티브', '11,875', '15', '0.13', '59', '500,000', '0.00'],
 ['2108',
  'KB 인버스 2X 금 선물 ETN(H)',
  '11,875',
  '20',
  '0.17',
  '59',
  '500,000',
  '0.00'],
 ['2109', 'PLUS 태양광&ESS', '9,885', '55', '0.56', '59', '600,000', '0.00'],
 ['2110', 'RISE 우량업종대표주', '11,830', '20', '0.17', '59', '500,000', '0.00'],
 ['2111', 'HANARO KRX300', '16,900', '', '0.00', '59', '350,000', '0.00'],
 ['2112',
  'ACE 일본TOPIX인버스(합성 H)',
  '2,5

In [84]:
len(bs.select('div.box_contents')[0].select('tr')[0].select('th'))

8

In [ ]:

for page in range(1,10):
    driver.find_element(By.XPATH, f'/html/body/div/div[4]/div[2]/div[1]/div[2]/div[2]/div/div/a[{page}]').click()
    a = bs.select('span.time')

In [53]:
a = bs.select('span.time')[0].text
c = bs.select('span.num')[0].text.strip()
d = bs.select('tr.first td span')[2].text.strip().replace('▲', '')
e = bs.select('tr.first td span')[3].text.strip().replace('+', '').replace('-', '').replace('%', '')
f = bs.select('tr.first td span')[4].text.strip()
e = bs.select('tr.first td span')
f



'4,740,007'

##### Send Keys
```python
driver.find_element_by_xpath(xpath).send_keys(id)
driver.find_element_by_xpath(xpath).send_keys(password)
driver.find_element_by_xpath(xpath).click()
```

외에도 Keys 내의 다양한 커맨드 입력 가능
```
from selenium.webdriver.common.keys import Keys
```
|Command          |Action |
|-----------------|-------|
|Keys.ENTER       |엔터    |
|Keys.RRETURN     |       |
|Keys.SPACE       |스페이스 |
|Keys.ARROW_UP    |화살표   |
|Keys.ARROW_DOWN  |       |
|Keys.ARROW_LEFT  |       |
|Keys.ARROW_RIGHT |       |
|Keys.BACK_SPACE  |지우기   |
|Keys.DELETE      |       |
|Keys.CONTROL     |Ctrl   |
|Keys.ALT         |Alt    |
|Keys.SHIFT       |Shift  |
|Keys.TAB         |Tab    |
|Keys.PAGE_UP     |Page-Up|
|Keys.PAGE_DOWN   |Page-Down|
|Keys.TAB         |Tab    |
|Keys.F1 ~ Keys.F9|F1~F9  |
|Keys.ESCAPE      |ESC    |
|Keys.HOME        |Home   |
|Keys.INSERT      |Insert |
|...              |...    |

In [30]:
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys('월급')
driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input').send_keys(Keys.ENTER)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[1]/div/div[3]/div/div/form/fieldset/div/input"}
  (Session info: chrome=127.0.6533.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00B28923+23283]
	(No symbol) [0x00AEE934]
	(No symbol) [0x00A20733]
	(No symbol) [0x00A6326F]
	(No symbol) [0x00A634AB]
	(No symbol) [0x00A9EE42]
	(No symbol) [0x00A84464]
	(No symbol) [0x00A9CB8D]
	(No symbol) [0x00A841B6]
	(No symbol) [0x00A58017]
	(No symbol) [0x00A5890D]
	GetHandleVerifier [0x00C1A5F3+1013699]
	GetHandleVerifier [0x00C23E4C+1052700]
	GetHandleVerifier [0x00C1D4B4+1025668]
	GetHandleVerifier [0x00B4EA2B+179195]
	(No symbol) [0x00AF6833]
	(No symbol) [0x00AF3198]
	(No symbol) [0x00AF3337]
	(No symbol) [0x00AEB4BE]
	BaseThreadInitThunk [0x769D7BA9+25]
	RtlInitializeExceptionChain

In [132]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [102]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
time.sleep(2)
bs = BeautifulSoup(driver.page_source, 'lxml')


In [37]:
driver.implicitly_wait(1)
driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[11]')

<selenium.webdriver.remote.webelement.WebElement (session="75cdb10dfd466b527eab1c4f1d86ecb6", element="f.91F91BE21EA5C32A30B0111A2FEC1D83.d.59DE45710F820107FC080537ECD5B4F9.e.2000")>

In [42]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[10]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')

[<td style="text-align: left;">202405</td>,
 <td style="text-align: left;">지에스</td>,
 <td style="text-align: left;">(유)가승개발</td>,
 <td>최광옥</td>,
 <td>2016/01/21</td>,
 <td>2016/01/21</td>,
 <td>R9112</td>,
 <td style="text-align: left;">골프장 및 스키장 운영업</td>,
 <td style="text-align: right;">0</td>,
 <td>2023/12/31</td>,
 <td>2024/03/27</td>,
 <td></td>,
 <td style="text-align: right;">49,495</td>,
 <td style="text-align: right;">52,072</td>,
 <td style="text-align: right;">-2,577</td>,
 <td style="text-align: right;">100</td>,
 <td style="text-align: right;">0</td>,
 <td style="text-align: right;">-3,863</td>,
 <td>일반회사</td>,
 <td>1345140003989</td>,
 <td>1198800340</td>,
 <td style="text-align: left;">202305</td>,
 <td style="text-align: left;">지에스</td>,
 <td style="text-align: left;">(유)가승개발</td>,
 <td>최광옥</td>,
 <td>2016/01/21</td>,
 <td>2016/04/01</td>,
 <td>R9112</td>,
 <td style="text-align: left;">골프장 및 스키장 운영업</td>,
 <td style="text-align: right;">0</td>,
 <td>2022/12/31</td>,
 <t

In [108]:
driver.get('https://www.egroup.go.kr/egps/wi/stat/kap/psitnCmpnySumryList.do')
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[10]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')

input_list = []
for i in range(10):
    input_list.append([i.text for i in bs.select('tbody tr')[i].select('td')[:8]])

for i in range(3, 12):
    driver.find_element(By.XPATH, f'/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[{i}]').click()
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[1]/table/tbody/tr[10]/td[3]")))
    bs = BeautifulSoup(driver.page_source, 'lxml')
    for i in range(10):
        input_list.append([i.text for i in bs.select('tbody tr')[i].select('td')[:8]])
    print(input_list)
    print()

# /html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[3] 1 2
# /html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[11] 9 10

input_list


[['202405', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/01/21', 'R9112', '골프장 및 스키장 운영업'], ['202305', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/04/01', 'R9112', '골프장 및 스키장 운영업'], ['202205', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/04/01', 'R91', '스포츠 및 오락관련 서비스업'], ['202405', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R9112', '골프장 및 스키장 운영업'], ['202305', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R9112', '골프장 및 스키장 운영업'], ['202205', '카카오', '(유)가승개발', '최광옥', '2016/01/21', '2021/02/01', 'R91', '스포츠 및 오락관련 서비스업'], ['202305', '중앙', '(유)그린에너지매니지먼트아시아', '최준용', '2021/05/24', '2021/06/01', 'L6811', '부동산 임대업'], ['202205', '에스케이', '(유)나눔푸드', '김치훈', '2008/05/27', '2022/05/01', 'C10', '식료품 제조업'], ['202405', '네이버', '(유)나라미', '최선규', '2022/01/14', '2024/04/01', 'K6499', '그 외 기타 금융업'], ['202405', '에스케이', '(유)로지소프트', '이영수', '2002/08/09', '2022/10/01', 'J5822', '시스템ㆍ응용 소프트웨어 개발 및 공급업'], ['202405', '지에스', '(유)가승개발', '최광옥', '2016/01/21', '2016/01/21', 'R9112', '골프장 및 스키장 운영업'

ElementClickInterceptedException: Message: element click intercepted: Element <a href="javascript:fn_pagination('...');">3</a> is not clickable at point (240, 623). Other element would receive the click: <div class="bgshadow"></div>
  (Session info: chrome=127.0.6533.120)
Stacktrace:
	GetHandleVerifier [0x00B28923+23283]
	(No symbol) [0x00AEE934]
	(No symbol) [0x00A20733]
	(No symbol) [0x00A691C7]
	(No symbol) [0x00A675D9]
	(No symbol) [0x00A654CB]
	(No symbol) [0x00A64AAF]
	(No symbol) [0x00A599EF]
	(No symbol) [0x00A8441C]
	(No symbol) [0x00A59485]
	(No symbol) [0x00A846B4]
	(No symbol) [0x00A9CB8D]
	(No symbol) [0x00A841B6]
	(No symbol) [0x00A58017]
	(No symbol) [0x00A5890D]
	GetHandleVerifier [0x00C1A5F3+1013699]
	GetHandleVerifier [0x00C23E4C+1052700]
	GetHandleVerifier [0x00C1D4B4+1025668]
	GetHandleVerifier [0x00B4EA2B+179195]
	(No symbol) [0x00AF6833]
	(No symbol) [0x00AF3198]
	(No symbol) [0x00AF3337]
	(No symbol) [0x00AEB4BE]
	BaseThreadInitThunk [0x769D7BA9+25]
	RtlInitializ

In [ ]:
# bs.select('table')[0].select('tr')[0] 이런식으로 모든 컬럼을 확인할 수 있다.

In [91]:
[i.text for i in bs.select('tbody tr')[1].select('td')[:8]]


['202305',
 '지에스',
 '(유)가승개발',
 '최광옥',
 '2016/01/21',
 '2016/04/01',
 'R9112',
 '골프장 및 스키장 운영업']

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

##### Moving between Windows

```python
previous_window = driver.window_handles[0]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
```


##### Moving

```python
driver.forward()
driver.back()
```

#### Waiting

Selenium은 실행 중인 driver를 이용하여 정보를 추출  
driver에 데이터 로딩이 완료되지 않을 경우 데이터 수집이 불가  
따라서 페이지 이동 등의 작업이 들어가면 다음 작업이 완료될 때까지 대기해야만 함

##### Time

가장 기본적인 방법으로 대기 시간을 time 모듈을 통해 명시

```python
import time
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.get(url="http://www.naver.com")
time.time(5)
driver.close()
```

##### Implicit Wait

위의 time에서는 2가지 문제가 존재
1. 모든 반응형 동작마다 time을 걸어 웹 로딩 대기
2. time을 5초로 설정했지만 실제 그보다 동작이 빨리 끝날 경우 불필요한 시간 대기

위의 문제를 해결하고자 implicit wait을 이용  
이는 driver의 옵션 설정으로 반응형 동작에서 최대 허용 대기 시간을 설정  
만약 허용 시간 이내에 로드가 완료될 경우 다음 작업을 바로 진행

```python
from selenium import webdriver


driver = webdriver.Chrome(executable_path='chromedriver')
driver.implicitly_wait(time_to_wait=5)
driver.get(url="http://www.naver.com")
driver.close()

```

##### Explicit Wait

implicit wait에서는 driver가 로딩을 대기하는 시간을 설정하여 5초를 설정하였어도 이전에 로드가 완료되면 다음 작업의 진행이 가능하였음  
하지만 위 경우도 문제가 있는데 5초가 지나도 내가 원하는 데이터가 로드가 안 되었을 경우 데이터를 수집할 수 없음  
따라서 특정 값이 로드될 때까지 기다렸다가 그 값이 로드되면 다음 작업을 진행할 필요성이 발생  
이 때 이용하는 게 explicit wait

```python
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path='chromedriver') 
driver.get(url="http://www.naver.com")

try:
    element = WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.CLASS_NAME , 'paging'))
    )
finally:
    driver.quit()

driver.close()

```

until: 조건이 False인 동안에 계속 실행  
not_until: 조건이 True인 동안에 계속 실행

#### ActionChains

연속 동작을 수행하기 위함

ex) control + c
```python
ActionChains(driver).key_down(Keys.CONTROL).send_keys('c').key_up(Keys.CONTROL).perform()
```

##### Drag and Drop

```python
from selenium.webdriver import ActionChains

element = driver.find_element(By.NAME, "source")
target = driver.find_element(By.NAME, "target")

action_chains = ActionChains(driver)
action_chains.drag_and_drop(element, target).perform()
```

#### Others

##### Options
```python
options = webdriver.ChromeOptions()
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(executable_path, options=options)
```

##### Alert

경고창 발생 시 이에 수락, 거절 등의 행동을 취할 수 있음

```python
from selenium.webdriver.common.alert import Alert

Alert(driver).accept() # 수락
Alert(driver).dismiss() # 거절
Alert(driver).send_keys(keysToSend=key) # 특정 키를 보낼 수 있음
```

##### Scroll Down
페이지의 최하단으로 이동
```python
driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
```
<br>

특정 태그가 등장할 때까지 이동
```python
from selenium.webdriver import ActionChains

some_tag = driver.find_element_by_id('gorio')
ActionChains(driver).move_to_element(some_tag).perform()
```

##### Minimize/Maximize

```python
driver.minimize_window()
driver.maximize_window()
```

##### Screen Shot
```python
driver.save_screenshot('screenshot.png')
```

In [135]:
# 인자값은 저장된 사진이름이다.

In [163]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


# Practice

In [164]:
driver.get('https://dart.fss.or.kr/')

driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
).send_keys('삼성전자')
driver.find_element(
    By.XPATH,
    '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[3]/a'
).send_keys(Keys.ENTER)

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'))
).find_element(By.XPATH,
    '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'
).click()




In [138]:
driver.window_handles
# /html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/ul/li[1]/a

['E2E911A047AA74E187083D35886A76E1', 'D1B8A3580506B8E299889307CB1453F0']

In [139]:
driver.switch_to.window(driver.window_handles[-1]) #드라이버가 제어하는 화면을 가장 최근에 뜬 화면으로 옮긴다.
driver.find_element(
    By.XPATH,
    '/html/body/div[3]/div/div[2]/div[1]/div[2]/ul/ul/li[5]/ul/li[2]/ul/li[1]/a'
).click()

bs = BeautifulSoup(driver.page_source, 'lxml')
bs.select('tavle')

#셀리늄에서 크롤링이 안되는게 Iframe이다.

[]

In [129]:
driver.quit() #전체다운
driver.close() #현 드라이버 다운

In [130]:
driver.switch_to.window(driver.window_handles[-1])

MaxRetryError: HTTPConnectionPool(host='localhost', port=52500): Max retries exceeded with url: /session/3bd91ef9d71182b01b90759ea30e819d/window/handles (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001D7B22F39E0>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [183]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))


In [191]:
response = requests.get('https://finance.naver.com/sise/sise_market_sum.naver', headers = {'user-agent': 'Mozilla 5.0'})
bs = BeautifulSoup(response.text, 'lxml')
name_list = []
for i in range(20):
    if i % 2 == 0:
        name_list.append(bs.select('tbody tr td a')[i].text)

name_list

driver.get('https://dart.fss.or.kr/')

for i in name_list:
    driver.find_element(By.XPATH, '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[2]/div/span[2]/input'
    ).send_keys(f'{i}')
    driver.find_element(
        By.XPATH,
        '/html/body/div[2]/div[1]/div[3]/div/div[2]/form/div[1]/div[3]/a'
    ).send_keys(Keys.ENTER)

    bs = BeautifulSoup(driver.page_source, 'lxml')
    text = bs.select('tr td a')[1].text.strip()
    if text == '반기보고서 (2024.06)':
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'))
                ).find_element(By.XPATH,
                '/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/form[3]/div/div[1]/table/tbody/tr[6]/td[3]/a'
                ).click()
        

    driver.switch_to.window(driver.window_handles[-1])
    
        
    # r_20240816000240
    # r_20240731000614
    # r_20240820000359

IndexError: list index out of range

In [200]:
bs = BeautifulSoup(driver.page_source, 'lxml')

bs.select('tr td a')[1].text.strip()

'주식등의대량보유상황보고서(일반)'

In [182]:
name_list


['삼성전자',
 'SK하이닉스',
 'LG에너지솔루션',
 '삼성바이오로직스',
 '현대차',
 '삼성전자우',
 '셀트리온',
 '기아',
 'KB금융',
 '신한지주']

In [ ]:

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div[3]/div[3]/div/form[2]/div/div[1]/div/div/div[2]/a[10]'))
)
bs = BeautifulSoup(driver.page_source, 'lxml')